# Q1

## Stochastic Processes

A stochastic process $\{ X_t : t \in T\}$ is a collection of random variables indexed by some set T. 

## Gaussian Processes

A kind of stochastic process where every finite colleciton of random variables $(X_{t_1}, X_{t_2}, ..., X_{t_n})$ has a multivariate normal distribution. 

Formally, a GP $\{f(x)\}$ is specified by: 
- A mean function $m(x) = \mathbb{E} [f(x)]$.
- A covariance function or kernel $k(x,x') = Cov[f(x),f(x')]$.

We write: $f(x) \sim GP(m(x), k(x,x'))$.

This is nice because now we can interpret the kernel as capturing how outputs x and x' co-vary. By choosing different kernels, we embed different assumptions about the function's smoothness, periodicity, etc. 

## Variational Inference
This is a method for approximating difficult to compute posterior distributions $p(\theta | x)$ in a Bayesian model. Instead of sampling like in MCMC, VI uses optimization. 
- We posit a simple family of distributions $Q$ that we will use as a stand-in for the complex true posterior. 
- We then find the member of this simple family $q*(\theta)$ that is closest (in terms of KL divergence) to the true posterior $p(\theta | x)$.

## Evidence Lower Bound (Elbo)

For a proposed variational distribution $q(\theta)$, $ELBO(q) = \mathbb{E}_{q(\theta)}[log(p(x,\theta)) - log(q(\theta))]$
- The VI procedure is to maximize $ELBO(q)$ over all $q \in Q$

In [1]:
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [4]:
cal_data = fetch_california_housing()
X = cal_data.data
y = cal_data.target

X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X, y, test_size=0.2, random_state=42)

n_small = 200
X_train_small = X_train_full[:n_small]
y_train_small = y_train_full[:n_small]

scaler = StandardScaler()
X_train_small_scaled = scaler.fit_transform(X_train_small)
X_test_scaled = scaler.transform(X_test_full)


In [5]:
with pm.Model() as gp_model:
    len_scale = pm.Gamma('len_scale', alpha=2, beta=1)
    eta = pm.HalfNormal('eta', sigma=2)

    #rbf kernel
    cov_func = eta**2 * pm.gp.cov.ExpQuad(input_dim=X_train_small.shape[1], ls=len_scale)
    mean_func = pm.gp.mean.Zero()

    gp = pm.gp.Marginal(mean_func=mean_func, cov_func=cov_func)
    sigma = pm.HalfNormal("sigma", sigma=2)

    y_obs = gp.marginal_likelihood("y_obs", X_train_small_scaled, y_train_small, noise=sigma)
    trace_gp = pm.sample(1000, tune=1000, 
                         chains=4, cores=1, target_accept=0.95)
    
print(az.summary(trace_gp, var_names=['len_scale', 'eta', 'sigma']))

/Users/aryanmisra/miniforge3/envs/pymc_env/lib/python3.12/site-packages/pymc/gp/gp.py:55: FutureWarning: The 'noise' parameter has been been changed to 'sigma' in order to standardize the GP API and will be deprecated in future releases.
  warnings.warn(_noise_deprecation_warning, FutureWarning)
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (4 chains in 1 job)
NUTS: [len_scale, eta, sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 179 seconds.


            mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
len_scale  5.838  1.257   3.741    8.297      0.030    0.021    1716.0   
eta        2.966  0.689   1.820    4.249      0.016    0.012    1759.0   
sigma      0.592  0.036   0.525    0.660      0.001    0.001    2007.0   

           ess_tail  r_hat  
len_scale    2432.0    1.0  
eta          2309.0    1.0  
sigma        2121.0    1.0  


In [7]:
post_means = trace_gp.posterior.mean(dim=["chain", "draw"])
point_dict = {
    "len_scale": float(post_means["len_scale"].values),
    "eta": float(post_means["eta"].values),
    "sigma": float(post_means["sigma"].values),
}
with gp_model:
    mu_pred, var_pred = gp.predict(
        X_test_scaled, point=point_dict,
        diag=True, pred_noise=True
    )

# 8) Evaluate MSE
mse_gp = np.mean((mu_pred - y_test_full)**2)
print("GP Test MSE (trained on 200 pts):", mse_gp)

GP Test MSE (trained on 200 pts): 0.4520293868467607


In [10]:
n_hidden = 50
n_features = X_train_small.shape[1]
with pm.Model() as bnn_small:
    W1 = pm.Normal('W1', mu=0, sigma=1, shape=(n_features, n_hidden))
    b1 = pm.Normal('b1', mu=0, sigma=1, shape=n_hidden)

    layer1 = pm.math.tanh(pm.math.dot(X_train_small_scaled, W1) + b1)

    W2 = pm.Normal('W2', mu=0, sigma=1, shape=(n_hidden,))
    b2 = pm.Normal('b2', mu=0, sigma=1)

    mu = pm.math.dot(layer1, W2) + b2
    sigma = pm.HalfNormal('sigma', sigma=1)

    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y_train_small)

    approx = pm.fit(n=12000, method="advi")
    trace_bnn_small = approx.sample(draws=2000)
print(az.summary(trace_bnn_small, var_names=['W1', 'b1', 'W2', 'b2', 'sigma']))

Output()

Finished [100%]: Average Loss = 520.76
arviz - WARNING - Shape validation failed: input_shape: (1, 2000), minimum_shape: (chains=2, draws=4)


           mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
W1[0, 0]  0.021  0.887  -1.467    1.875      0.020    0.015    2015.0   
W1[0, 1]  0.044  0.888  -1.659    1.649      0.020    0.014    1963.0   
W1[0, 2]  0.115  0.833  -1.322    1.822      0.018    0.013    2087.0   
W1[0, 3]  0.076  0.898  -1.585    1.791      0.020    0.014    1949.0   
W1[0, 4]  0.030  0.876  -1.599    1.657      0.020    0.015    1897.0   
...         ...    ...     ...      ...        ...      ...       ...   
W2[47]   -0.024  0.380  -0.733    0.701      0.009    0.006    1922.0   
W2[48]    0.044  0.379  -0.680    0.745      0.009    0.006    1907.0   
W2[49]    0.014  0.395  -0.721    0.754      0.009    0.006    2039.0   
b2        1.778  0.356   1.126    2.451      0.008    0.006    1933.0   
sigma     2.683  0.418   1.947    3.493      0.010    0.007    1860.0   

          ess_tail  r_hat  
W1[0, 0]    1841.0    NaN  
W1[0, 1]    1912.0    NaN  
W1[0, 2]    1879.0    NaN  
W1[0, 3]   

In [11]:
posterior_means_bnn_small = trace_bnn_small.posterior.mean(dim=["chain", "draw"])

W1_m = posterior_means_bnn_small['W1'].values
b1_m = posterior_means_bnn_small['b1'].values
W2_m = posterior_means_bnn_small['W2'].values
b2_m = float(posterior_means_bnn_small['b2'].values)

def nn_predict(X_):
    layer1_ = np.tanh(X_ @ W1_m + b1_m)
    return layer1_ @ W2_m + b2_m

y_pred_small_bnn = nn_predict(X_test_scaled)
mse_small_bnn = np.mean((y_pred_small_bnn - y_test_full)**2)
print("Small BNN Test MSE (trained on 200 pts):", mse_small_bnn)

Small BNN Test MSE (trained on 200 pts): 1.138061048540759


In [14]:
scaler_full = StandardScaler()
X_train_full_scaled = scaler_full.fit_transform(X_train_full)
X_test_full_scaled = scaler_full.transform(X_test_full)

n_hidden_1 = 20
n_hidden_2 = 20

with pm.Model() as bnn_large:
    W1 = pm.Normal('W1', mu=0, sigma=1, shape=(n_features, n_hidden_1))
    b1 = pm.Normal('b1', mu=0, sigma=1, shape=(n_hidden_1,))

    layer1 = pm.math.tanh(pm.math.dot(X_train_full_scaled, W1) + b1)

    W2 = pm.Normal('W2', mu=0, sigma=1, shape=(n_hidden_1, n_hidden_2))
    b2 = pm.Normal('b2', mu=0, sigma=1, shape=(n_hidden_2,))

    layer2 = pm.math.tanh(pm.math.dot(layer1, W2) + b2)

    W3 = pm.Normal('W3', mu=0, sigma=1, shape=(n_hidden_2,))
    b3 = pm.Normal('b3', mu=0, sigma=1)

    mu = pm.math.dot(layer2, W3) + b3
    sigma = pm.HalfNormal('sigma', sigma=1)

    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y_train_full)

    approx_large = pm.fit(n=20000, method="advi")
    trace_bnn_large = approx_large.sample(draws=2000)
print(az.summary(trace_bnn_large, var_names=['W1', 'b1', 'W2', 'b2', 'W3', 'b3', 'sigma']))


Output()

Finished [100%]: Average Loss = 27,513
arviz - WARNING - Shape validation failed: input_shape: (1, 2000), minimum_shape: (chains=2, draws=4)


           mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
W1[0, 0] -0.082  0.656  -1.307    1.129      0.015    0.013    1789.0   
W1[0, 1]  0.018  0.681  -1.298    1.267      0.015    0.011    2127.0   
W1[0, 2]  0.030  0.669  -1.173    1.280      0.015    0.011    1942.0   
W1[0, 3]  0.076  0.725  -1.278    1.435      0.016    0.011    1995.0   
W1[0, 4]  0.013  0.649  -1.254    1.177      0.014    0.010    2141.0   
...         ...    ...     ...      ...        ...      ...       ...   
W3[17]    0.013  0.131  -0.243    0.255      0.003    0.002    2023.0   
W3[18]   -0.017  0.140  -0.273    0.242      0.003    0.002    1669.0   
W3[19]   -0.001  0.133  -0.258    0.240      0.003    0.002    2034.0   
b3        2.068  0.109   1.886    2.297      0.003    0.002    1727.0   
sigma     1.268  0.061   1.154    1.385      0.001    0.001    1919.0   

          ess_tail  r_hat  
W1[0, 0]    1377.0    NaN  
W1[0, 1]    2011.0    NaN  
W1[0, 2]    1816.0    NaN  
W1[0, 3]   

In [15]:
post_mean_bnn_large = trace_bnn_large.posterior.mean(dim=["chain", "draw"])

W1_m = post_mean_bnn_large['W1'].values
b1_m = post_mean_bnn_large['b1'].values
W2_m = post_mean_bnn_large['W2'].values
b2_m = post_mean_bnn_large['b2'].values
W3_m = post_mean_bnn_large['W3'].values
b3_m = float(post_mean_bnn_large['b3'].values)

def forward_large_nn(X_):
    layer1_ = np.tanh(X_ @ W1_m + b1_m)
    layer2_ = np.tanh(layer1_ @ W2_m + b2_m)
    return layer2_ @ W3_m + b3_m

y_pred_large = forward_large_nn(X_test_full_scaled)
mse_bnn_large = np.mean((y_pred_large - y_test_full)**2)
print("Large BNN Test MSE (trained on full data):", mse_bnn_large)

Large BNN Test MSE (trained on full data): 1.3101384420820532
